In [1]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime
import time
from scipy import stats
import numpy as np
import matplotlib as plt

import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

%matplotlib inline

pd.set_option('display.max_rows', 10000)

In [2]:
TRAINING_ROUND = 'Training_Round'
EPOCHS = 'Epochs'
SCENARIO = 'Scenario'
EMBEDDING = 'Embedding'
TAGGING_SCHEME = 'Tagging_Scheme'
CAP_DIM = 'Capitalisation_Dim'
LOWER = 'Lower'
CHAR_DIM = 'Char_LSTM_Dim'
WORD_DIM = 'Word_LSTM_Dim'
PRECISION = 'Precision'
RECALL = 'Recall'
F1_SCORE = 'F1_Score'

In [3]:
columns = [TRAINING_ROUND, EPOCHS, SCENARIO, EMBEDDING, TAGGING_SCHEME, CAP_DIM, LOWER, CHAR_DIM, WORD_DIM, PRECISION, RECALL, F1_SCORE]

In [4]:
columns

['Training_Round',
 'Epochs',
 'Scenario',
 'Embedding',
 'Tagging_Scheme',
 'Capitalisation_Dim',
 'Lower',
 'Char_LSTM_Dim',
 'Word_LSTM_Dim',
 'Precision',
 'Recall',
 'F1_Score']

In [5]:
pd.set_option('display.max_columns', 1000)
training_data_df = pd.read_csv('resultados_sem_outliers.csv')
training_data_df 

,Unnamed: 0,Training_Round,Epochs,Scenario,Embedding,Tagging_Scheme,Capitalisation_Dim,Lower,Char_LSTM_Dim,Word_LSTM_Dim,Precision,Recall,F1_Score
0,320,0,5,selective,FastText,IOB2,False,False,25,100,58.83,56.51,57.64
1,352,1,5,selective,FastText,IOB2,False,False,25,100,64.86,50.77,56.96
2,384,2,5,selective,FastText,IOB2,False,False,25,100,63.22,51.77,56.93
3,416,3,5,selective,FastText,IOB2,False,False,25,100,59.05,56.76,57.88
4,448,4,5,selective,FastText,IOB2,False,False,25,100,58.87,55.11,56.93
5,480,5,5,selective,FastText,IOB2,False,False,25,100,59.32,48.73,53.50
6,512,6,5,selective,FastText,IOB2,False,False,25,100,59.65,50.99,54.98
7,544,7,5,selective,FastText,IOB2,False,False,25,100,63.90,50.59,56.47
8,576,8,5,selective,FastText,IOB2,False,False,25,100,59.73,56.47,58.05
9,608,9,5,selective,FastText,IOB2,False,False,25,100,62.26,55.36,58.61


In [6]:
training_data_df = training_data_df[training_data_df.Lower == True]
len(training_data_df)

581

In [7]:
training_data_df.groupby([TAGGING_SCHEME]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Tagging_Scheme,,,,,,,,
IOB2,292.0,60.648116,7.139823,44.34,52.115,63.765,65.83,68.26
IOBES,289.0,60.229516,7.452188,42.97,50.500,63.590,65.86,67.99


In [8]:
training_data_df.groupby([EMBEDDING]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Embedding,,,,,,,,
Wang2Vec,141.0,66.504043,0.848442,63.18,66.10,66.48,67.0100,68.26
FastText,140.0,65.011429,1.225288,56.80,64.45,65.18,65.7225,66.90
Glove,145.0,62.602000,1.003969,59.96,61.90,62.70,63.3100,64.89
Word2Vec,155.0,48.771742,1.816029,42.97,47.79,49.00,49.9650,52.42


In [9]:
training_data_df.groupby([CAP_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Capitalisation_Dim,,,,,,,,
True,295.0,60.599627,7.315023,43.81,51.325,63.76,66.085,68.26
False,286.0,60.275140,7.280542,42.97,51.505,63.48,65.680,68.07


In [10]:
training_data_df.groupby([CHAR_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Char_LSTM_Dim,,,,,,,,
50,288.0,60.456979,7.206507,45.04,51.515,63.665,65.93,68.07
25,293.0,60.423106,7.390465,42.97,51.410,63.600,65.75,68.26


In [11]:
training_data_df.groupby([WORD_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Word_LSTM_Dim,,,,,,,,
100,290.0,60.642345,7.248220,42.97,51.5075,63.88,66.0875,68.26
200,291.0,60.238144,7.345432,43.81,51.3500,63.55,65.6900,68.07


In [12]:
training_data_df.groupby([EMBEDDING, TAGGING_SCHEME, CAP_DIM, WORD_DIM, CHAR_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

count  \
Embedding Tagging_Scheme Capitalisation_Dim Word_LSTM_Dim Char_LSTM_Dim          
Wang2Vec  IOB2           False              100           50              10.0   
                         True               100           25               9.0   
                         False              200           50               6.0   
          IOBES          True               200           25              10.0   
                                            100           50               9.0   
          IOB2           True               100           50              10.0   
                                            200           50               9.0   
          IOBES          True               100           25              10.0   
          IOB2           True               200           25               7.0   
          IOBES          False              100           50               7.0   
                         True               200           50              10.0   
                         False              200           50               9.0   
                                            100           25               9.0   
          IOB2           False              200           25               9.0   
                                            100           25               8.0   
          IOBES          False              200           25               9.0   
FastText  IOB2           True               100           25              10.0   
                                                          50               9.0   
          IOBES          False              100           25               9.0   
                         True               200           50               8.0   
          IOB2           True               200           25               9.0   
                         False              100           25              10.0   
          IOBES          True               100           25               7.0   
                         False              100           50               8.0   
          IOB2           False              100           50               8.0   
          IOBES          False              200           25               9.0   
                         True               200           25              10.0   
          IOB2           False              200           50              10.0   
                                                          25               8.0   
                         True               200           50               9.0   
          IOBES          True               100           50               8.0   
                         False              200           50               8.0   
Glove     IOBES          False              200           50               8.0   
          IOB2           True               100           50              10.0   
          IOBES          True               100           25               9.0   
          IOB2           True               100           25               9.0   
          IOBES          True               200           50               9.0   
          IOB2           False              100           25               9.0   
                                            200           50               9.0   
          IOBES          True               200           25               9.0   
          IOB2           False              100           50              10.0   
                         True               200           50               9.0   
          IOBES          True               100           50               9.0   
                         False              100           25               8.0   
                                                          50               8.0   
                                            200           25               9.0   
          IOB2           False              200           25              10.0   
                         True               200        

In [13]:
training_data_df.groupby([TRAINING_ROUND]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Training_Round,,,,,,,,
4,52.0,60.984231,6.935010,47.28,58.4900,64.165,65.6900,68.26
2,61.0,60.732951,7.449083,44.08,52.4200,64.270,66.2300,68.07
9,56.0,60.657679,7.146791,46.94,52.1925,63.655,65.8700,67.87
8,60.0,60.625833,6.869073,46.61,58.7675,63.620,65.5025,67.73
1,60.0,60.601667,7.236562,46.26,51.4500,63.400,66.2200,67.99
5,58.0,60.431207,7.341524,47.59,50.5700,63.690,66.1125,67.86
6,60.0,60.307667,7.453801,45.04,50.6950,63.575,65.6475,67.74
3,59.0,60.250169,7.642747,44.33,50.3100,63.610,65.8550,67.79
0,61.0,60.226721,7.475321,42.97,51.5000,63.180,65.8600,67.57


In [14]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [15]:
rp.summary_cont(training_data_df[F1_SCORE])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,F1_Score,581.0,60.439897,7.293584,0.302589,59.845593,61.0342


In [16]:
rp.summary_cont(training_data_df.groupby(EMBEDDING))[F1_SCORE]

,N,Mean,SD,SE,95% Conf.,Interval
Embedding,,,,,,
FastText,140,65.011429,1.225288,0.103556,64.808459,65.214398
Glove,145,62.602000,1.003969,0.083375,62.438585,62.765415
Wang2Vec,141,66.504043,0.848442,0.071452,66.363997,66.644088
Word2Vec,155,48.771742,1.816029,0.145867,48.485843,49.057641


In [17]:
print(F1_SCORE, EMBEDDING, TAGGING_SCHEME, CAP_DIM, CHAR_DIM, WORD_DIM)

F1_Score Embedding Tagging_Scheme Capitalisation_Dim Char_LSTM_Dim Word_LSTM_Dim


In [18]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('F1_Score ~ C(Embedding)*C(Tagging_Scheme)*C(Capitalisation_Dim)*C(Char_LSTM_Dim)*C(Word_LSTM_Dim)', training_data_df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .20f}")

Overall model F( 63, 517) =  293.305, p =  0.00000000000000000000


In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               F1_Score   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                     293.3
Date:                Wed, 10 Apr 2019   Prob (F-statistic):               0.00
Time:                        21:57:01   Log-Likelihood:                -931.41
No. Observations:                 581   AIC:                             1991.
Df Residuals:                     517   BIC:                             2270.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================================================================================
                                                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                           65.2590      0.403    161.923      0.000      64.467      66.051
C(Embedding)[T.Glove]                                                                                                               -2.5534      0.586     -4.361      0.000      -3.704      -1.403
C(Embedding)[T.Wang2Vec]                                                                                                             0.9597      0.605      1.588      0.113      -0.228       2.147
C(Embedding)[T.Word2Vec]                                                                                                           -15.6220      0.570    -27.409      0.000     -16.742     -14.502
C(Tagging_Scheme)[T.IOBES]                                                                                                           0.4343      0.586      0.742      0.459      -0.716       1.585
C(Capitalisation_Dim)[T.True]                                                                                                        0.5770      0.570      1.012      0.312      -0.543       1.697
C(Char_LSTM_Dim)[T.50]                                                                                                              -0.3015      0.605     -0.499      0.618      -1.489       0.886
C(Word_LSTM_Dim)[T.200]                                                                                                             -0.6402      0.605     -1.059      0.290      -1.828       0.547
C(Embedding)[T.Glove]:C(Tagging_Scheme)[T.IOBES]                                                                                    -0.7099      0.852     -0.833      0.405      -2.384       0.965
C(Embedding)[T.Wang2Vec]:C(Tagging_Scheme)[T.IOBES]                                                                                 -0.4031      0.852     -0.473      0.636      -2.077       1.271
C(Embedding)[T.Word2Vec]:C(Tagging_Scheme)[T.IOBES]                                                                                 -2.1333      0.817     -2.611      0.009      -3.739      -0.528
C(Embedding)[T.Glove]:C(Capitalisation_Dim)[T.True]                                                                                 -0.4659      0.828     -0.563      0.574      -2.093       1.161
C(Embedding)[T.Wang2Vec]:C(Capitalisation_Dim)[T.True]                                                                          

In [20]:
res = sm.stats.anova_lm(model, typ= 2)

In [28]:
type(res)

pandas.core.frame.DataFrame

In [31]:
res.to_csv('anova.csv', float_format='%.10f', columns=['PR(>F)'])

In [21]:
res

,sum_sq,df,F,PR(>F)
C(Embedding),29805.623696,3.0,6116.625250,0.000000
C(Tagging_Scheme),4.438826,1.0,2.732770,0.098916
C(Capitalisation_Dim),2.147286,1.0,1.321980,0.250769
C(Char_LSTM_Dim),4.046514,1.0,2.491242,0.115093
C(Word_LSTM_Dim),15.681142,1.0,9.654118,0.001993
C(Embedding):C(Tagging_Scheme),4.169763,3.0,0.855707,0.463970
C(Embedding):C(Capitalisation_Dim),3.084378,3.0,0.632967,0.594003
C(Tagging_Scheme):C(Capitalisation_Dim),0.033966,1.0,0.020911,0.885078
C(Embedding):C(Char_LSTM_Dim),14.835709,3.0,3.044542,0.028470
C(Tagging_Scheme):C(Char_LSTM_Dim),2.219961,1.0,1.366722,0.242915


In [22]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(Embedding),29805.623696,9935.207899,3.0,6116.625250,0.000000,0.968750,9.685403e-01
C(Tagging_Scheme),4.438826,4.438826,1.0,2.732770,0.098916,0.000144,9.147373e-05
C(Capitalisation_Dim),2.147286,2.147286,1.0,1.321980,0.250769,0.000070,1.699747e-05
C(Char_LSTM_Dim),4.046514,4.046514,1.0,2.491242,0.115093,0.000132,7.872338e-05
C(Word_LSTM_Dim),15.681142,15.681142,1.0,9.654118,0.001993,0.000510,4.568550e-04
C(Embedding):C(Tagging_Scheme),4.169763,1.389921,3.0,0.855707,0.463970,0.000136,-2.285190e-05
C(Embedding):C(Capitalisation_Dim),3.084378,1.028126,3.0,0.632967,0.594003,0.000100,-5.812749e-05
C(Tagging_Scheme):C(Capitalisation_Dim),0.033966,0.033966,1.0,0.020911,0.885078,0.000001,-5.168657e-05
C(Embedding):C(Char_LSTM_Dim),14.835709,4.945236,3.0,3.044542,0.028470,0.000482,3.237970e-04
C(Tagging_Scheme):C(Char_LSTM_Dim),2.219961,2.219961,1.0,1.366722,0.242915,0.000072,1.935944e-05


In [52]:
TUKEY_COLUMNS = ['Group 1', 'Group 2', 'Mean Difference', 'Lower', 'Upper', 'Reject?']
def generate_tukeyhsd_results(group):
    mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[group])
    mc_results = mc.tukeyhsd()
    print(mc_results)
    df = pd.DataFrame(data=mc_results._results_table.data[1:], columns=mc_results._results_table.data[0])
    df.columns = TUKEY_COLUMNS
    df.to_csv('tukey_' + group.lower() + '.csv', float_format='%.5f', index=False)

In [53]:
generate_tukeyhsd_results(EMBEDDING)

 Multiple Comparison of Means - Tukey HSD,FWER=0.05
 group1   group2  meandiff  lower    upper   reject
---------------------------------------------------
FastText  Glove   -2.4094  -2.8037  -2.0151   True 
FastText Wang2Vec  1.4926   1.0956   1.8897   True 
FastText Word2Vec -16.2397 -16.6277 -15.8517  True 
 Glove   Wang2Vec  3.902    3.5085   4.2956   True 
 Glove   Word2Vec -13.8303 -14.2147 -13.4458  True 
Wang2Vec Word2Vec -17.7323 -18.1196 -17.345   True 
---------------------------------------------------


In [54]:
generate_tukeyhsd_results(CAP_DIM)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff lower  upper reject
------------------------------------------
False   True   0.3245  -0.865 1.514 False 
------------------------------------------


In [55]:
generate_tukeyhsd_results(TAGGING_SCHEME)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 IOB2  IOBES  -0.4186  -1.6078 0.7706 False 
--------------------------------------------


In [56]:
generate_tukeyhsd_results(CHAR_DIM)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  25     50    0.0339  -1.1558 1.2236 False 
--------------------------------------------


In [57]:
generate_tukeyhsd_results(WORD_DIM)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper reject
-------------------------------------------
 100    200   -0.4042  -1.5934 0.785 False 
-------------------------------------------
